In [ ]:
# not raw data

In [9]:
import os
from parse_soft import ParseSoft
from search_soft import SearchSoft
from enrich_soft import EnrichSoft
from label_data import LabelData

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
# step 1: set defaults
geo = "GSE147066"
soft_dir = '/home/yuan/data'
meta_dir = '../data'
download_dir = '/home/yuan/rawdata/SRR'

In [18]:
# step 2: parse soft, and enrich

# analyze soft file
parser = ParseSoft(soft_dir)
soft_path = parser.soft_local_path(geo)
print('Retrieve meta data from soft file: ', soft_path)
softer = SearchSoft(soft_path)
data = softer.filter_data(softer.parse_rows)

# save to json
LabelData(meta_dir).save(data)

Retrieve meta data from soft file:  /home/yuan/data/ftp.ncbi.nlm.nih.gov/geo/series/GSE147nnn/GSE147066/soft/GSE147066_family.soft.gz
Save meta data into json:  /home/yuan/bio/scrnaseq_reference/data/labels/GSE147nnn/GSE147066/metadata.json


In [19]:
# step 3a: enrich data
data = LabelData(meta_dir).get_meta(geo)
enriched = EnrichSoft(data)()
LabelData(meta_dir).save(enriched)

Save meta data into json:  /home/yuan/bio/scrnaseq_reference/data/labels/GSE147nnn/GSE147066/metadata.json


In [20]:
# step 3b: label samples manually
from label_sample import LabelSample

def collator(sample):
    c = LabelSample(sample)
    c._protocol('scrna-seq', '10x3v2')
    if c.title.endswith('C'):
        c._healthy_control()
    elif c.title.endswith('I'):
        c._disease_patient('idiopathic Pulmonary Fibrosis')
    
    
data = LabelData(meta_dir).get_meta(geo)
for sample_id, sample in data['samples'].items():
    collator(sample)
LabelData(meta_dir).save(data)

Save meta data into json:  /home/yuan/bio/scrnaseq_reference/data/labels/GSE147nnn/GSE147066/metadata.json


In [21]:
for s in data['samples'].values():
    print(s['labels'])

{'disease': 'healthy', 'tissue': 'whole lung dissociate; collagen expressing mesenchymal cells only', 'group': 'control'}
{'disease': 'healthy', 'tissue': 'whole lung dissociate; collagen expressing mesenchymal cells only', 'group': 'control'}
{'disease': 'healthy', 'tissue': 'whole lung dissociate; collagen expressing mesenchymal cells only', 'group': 'control'}
{'disease': 'healthy', 'tissue': 'whole lung dissociate; collagen expressing mesenchymal cells only', 'group': 'control'}
{'disease': 'healthy', 'tissue': 'whole lung dissociate; collagen expressing mesenchymal cells only', 'group': 'control'}
{'disease': 'healthy', 'tissue': 'whole lung dissociate; collagen expressing mesenchymal cells only', 'group': 'control'}
{'disease': 'healthy', 'tissue': 'whole lung dissociate; collagen expressing mesenchymal cells only', 'group': 'control'}
{'disease': 'healthy', 'tissue': 'whole lung dissociate; collagen expressing mesenchymal cells only', 'group': 'control'}
{'disease': 'healthy', '

In [22]:
# step 4a: load some data
from utils import Utils

# data are determined previously
samn_srr = Utils.from_json('../results/samn_srr.json')
print(len(samn_srr))

47


In [24]:
# step 4b: Given BioSample, parse SRR accessions into samples
from parse_sra import ParseSra
from label_data import LabelData

data = LabelData(meta_dir).get_meta(geo)
enriched = ParseSra.parse_srr(data, samn_srr)
LabelData(meta_dir).save(enriched)

biosamples = 58, SRR accessions = 0.
Save meta data into json:  /home/yuan/bio/scrnaseq_reference/data/labels/GSE147nnn/GSE147066/metadata.json


In [25]:
# step 5: merge srr_urls into data

# sample.<sample_id>.SRR.<SRR accession>."ftp.sra.ebi.ac.uk"
file_name = 'srr_fastq_urls_simple.json'
urls_json = os.path.join('../results', file_name)
urls = Utils.from_json(urls_json)

# parse urls
data = LabelData(meta_dir).get_meta(geo)
enriched = ParseSra.parse_srr_urls(data, urls)
LabelData(meta_dir).save(enriched)

{'srr': 58, 'available': 0, 'unknown': 0, 'updated': 0}
Save meta data into json:  /home/yuan/bio/scrnaseq_reference/data/labels/GSE147nnn/GSE147066/metadata.json


In [26]:
# step 6a: download raw data using biosample accession using nextflow
from create_config import CreateConfig
from label_data import LabelData

sample_iter = LabelData(meta_dir).geo_sample_iter(geo)
# create bash script for downloading
CreateConfig(download_dir, geo).fetch_biosample(sample_iter)
# run the bash script showed here
# Outcome: nf download failed

Number of biosamples:  58
Number of biosamples with SRR:  0
bash /home/yuan/rawdata/SRR/GSE147066/fetch1.sh


In [27]:
# step 7: parse path of local fastq.gz with SRR in GEO data
# Note: run the process multiple times to make sure all SRRs are accessed
# print all samples if not any local *.fastq.gz is parsed
from parse_sra import ParseSra
from label_data import LabelData

data = LabelData(meta_dir).get_meta(geo)
data, unparsing = ParseSra.parse_local_fastq(data, download_dir)
if unparsing:
    print(geo, len(unparsing), unparsing[:6])
LabelData(meta_dir).save(data)

Save meta data into json:  /home/yuan/bio/scrnaseq_reference/data/labels/GSE147nnn/GSE147066/metadata.json
